# 支付与额度系统调试笔记本

本笔记本用于调试和测试 Mock 支付网关功能：
- 用户额度管理
- 订阅计划
- 支付流程模拟
- 每日额度重置

In [ ]:
# 添加项目根目录到路径
import sys
import os
sys.path.insert(0, os.path.dirname(os.getcwd()))

# 设置必要的环境变量
os.environ.setdefault("TELEGRAM_BOT_TOKEN", "test_token")
os.environ.setdefault("OPENAI_API_KEY", "test_key")

## 1. 导入支付模块

In [ ]:
from src.payment.mock_gateway import (
    MockPaymentGateway,
    SubscriptionTier,
    PaymentStatus,
    MockPayment,
    UserQuota,
    get_mock_payment_gateway
)

print("✅ 支付模块导入成功")

## 2. 创建支付网关实例

In [ ]:
# 创建支付网关
gateway = MockPaymentGateway()

print("Mock 支付网关创建成功")

## 3. 查看订阅计划

In [ ]:
# 获取所有订阅计划
plans = gateway.get_subscription_plans()

print("可用的订阅计划:")
print("=" * 50)

for tier, plan in plans.items():
    print(f"\n📦 {plan['name']} ({tier})")
    print(f"   价格: ¥{plan['price']}/月")
    print(f"   消息额度: {plan['messages_limit']} 条/天")
    print(f"   图片额度: {plan['images_limit']} 张/天")
    print(f"   功能: {', '.join(plan['features'])}")

## 4. 用户额度管理

In [ ]:
# 获取新用户的额度（自动创建免费用户）
user_id = "user_001"
quota = gateway.get_user_quota(user_id)

print(f"用户 {user_id} 的额度信息:")
print(f"  订阅等级: {quota.tier.value}")
print(f"  消息已用: {quota.messages_used}/{quota.messages_limit}")
print(f"  图片已用: {quota.images_used}/{quota.images_limit}")
print(f"  重置时间: {quota.reset_date}")

In [ ]:
# 检查额度
has_message_quota = gateway.check_quota(user_id, "message")
has_image_quota = gateway.check_quota(user_id, "image")

print(f"\n额度检查:")
print(f"  消息额度: {'✅ 有额度' if has_message_quota else '❌ 已用完'}")
print(f"  图片额度: {'✅ 有额度' if has_image_quota else '❌ 已用完'}")

In [ ]:
# 消费额度
print("\n消费额度测试:")

for i in range(5):
    success = gateway.consume_quota(user_id, "message", 1)
    quota = gateway.get_user_quota(user_id)
    status = "✅ 成功" if success else "❌ 失败"
    print(f"  消费 {i+1}: {status} (剩余: {quota.messages_limit - quota.messages_used})")

In [ ]:
# 消费超过额度
quota = gateway.get_user_quota(user_id)
remaining = quota.messages_limit - quota.messages_used

print(f"\n剩余额度: {remaining}")
print(f"尝试消费 {remaining + 5} 条消息:")

success = gateway.consume_quota(user_id, "message", remaining + 5)
print(f"  结果: {'✅ 成功' if success else '❌ 失败（额度不足）'}")

## 5. 支付流程模拟

In [ ]:
# 创建支付订单
user_id = "user_002"

payment = gateway.create_payment(
    user_id=user_id,
    tier=SubscriptionTier.PREMIUM,
    duration_days=30
)

print("创建支付订单:")
print(f"  订单 ID: {payment.payment_id}")
print(f"  用户 ID: {payment.user_id}")
print(f"  金额: ¥{payment.amount}")
print(f"  订阅等级: {payment.tier.value}")
print(f"  时长: {payment.duration_days} 天")
print(f"  状态: {payment.status.value}")

In [ ]:
# 支付前检查用户额度
quota_before = gateway.get_user_quota(user_id)

print(f"\n支付前用户额度:")
print(f"  订阅等级: {quota_before.tier.value}")
print(f"  消息限额: {quota_before.messages_limit}")
print(f"  图片限额: {quota_before.images_limit}")

In [ ]:
# 完成支付（Mock 直接成功）
success = gateway.complete_payment(payment.payment_id)

print(f"\n完成支付: {'✅ 成功' if success else '❌ 失败'}")
print(f"支付状态: {payment.status.value}")

In [ ]:
# 支付后检查用户额度
quota_after = gateway.get_user_quota(user_id)

print(f"\n支付后用户额度:")
print(f"  订阅等级: {quota_after.tier.value}")
print(f"  消息限额: {quota_after.messages_limit}")
print(f"  图片限额: {quota_after.images_limit}")
print(f"  订阅到期: {quota_after.subscription_end}")

print(f"\n变化:")
print(f"  消息限额: {quota_before.messages_limit} → {quota_after.messages_limit}")
print(f"  图片限额: {quota_before.images_limit} → {quota_after.images_limit}")

## 6. 查看支付记录

In [ ]:
# 获取用户的支付记录
payments = gateway.get_user_payments(user_id)

print(f"用户 {user_id} 的支付记录:")
for p in payments:
    print(f"\n  订单 {p.payment_id[:8]}...")
    print(f"    金额: ¥{p.amount}")
    print(f"    订阅: {p.tier.value}")
    print(f"    状态: {p.status.value}")
    print(f"    时间: {p.created_at}")

In [ ]:
# 获取单个支付记录
retrieved = gateway.get_payment(payment.payment_id)

if retrieved:
    print(f"获取支付记录:")
    print(f"  订单 ID: {retrieved.payment_id}")
    print(f"  完成时间: {retrieved.completed_at}")

## 7. 订阅过期测试

In [ ]:
from datetime import timedelta

# 模拟订阅过期
user_id = "user_003"

# 先升级用户
payment = gateway.create_payment(user_id, SubscriptionTier.BASIC, 30)
gateway.complete_payment(payment.payment_id)

quota = gateway.get_user_quota(user_id)
print(f"升级后:")
print(f"  订阅等级: {quota.tier.value}")
print(f"  消息限额: {quota.messages_limit}")

# 手动设置订阅为已过期
quota.subscription_end = quota.subscription_end - timedelta(days=60)

# 重新获取额度（会触发过期检查）
quota = gateway.get_user_quota(user_id)
print(f"\n过期后:")
print(f"  订阅等级: {quota.tier.value}")
print(f"  消息限额: {quota.messages_limit}")

## 8. 统计信息

In [ ]:
# 获取统计信息
stats = gateway.get_stats()

print("支付网关统计:")
print("=" * 40)
print(f"  总用户数: {stats['total_users']}")
print(f"  总支付数: {stats['total_payments']}")
print(f"  完成支付数: {stats['completed_payments']}")
print(f"  总收入: ¥{stats['total_revenue']:.2f}")

print(f"\n按订阅等级用户分布:")
for tier, count in stats.get('users_by_tier', {}).items():
    print(f"  {tier}: {count} 人")

## 9. 额度信息导出

In [ ]:
# 导出用户额度信息
import json

quota = gateway.get_user_quota("user_002")
quota_data = quota.to_dict()

print("用户额度数据导出:")
print(json.dumps(quota_data, indent=2, ensure_ascii=False, default=str))

## 10. 完整支付流程演示

In [ ]:
# 完整演示：新用户 → 使用免费额度 → 升级订阅 → 使用高级额度

new_user = "demo_user"

print("=" * 50)
print("完整支付流程演示")
print("=" * 50)

# 步骤1: 新用户获取免费额度
print("\n📱 步骤1: 新用户注册")
quota = gateway.get_user_quota(new_user)
print(f"  订阅等级: {quota.tier.value}")
print(f"  消息额度: {quota.messages_limit}")

# 步骤2: 使用一些免费额度
print("\n💬 步骤2: 使用免费额度")
for i in range(3):
    gateway.consume_quota(new_user, "message")
quota = gateway.get_user_quota(new_user)
print(f"  已使用: {quota.messages_used}/{quota.messages_limit}")

# 步骤3: 创建订阅订单
print("\n💳 步骤3: 创建订阅订单")
payment = gateway.create_payment(new_user, SubscriptionTier.PREMIUM, 30)
print(f"  订单号: {payment.payment_id[:8]}...")
print(f"  金额: ¥{payment.amount}")

# 步骤4: 完成支付
print("\n✅ 步骤4: 完成支付")
gateway.complete_payment(payment.payment_id)
print(f"  支付状态: {payment.status.value}")

# 步骤5: 查看升级后的额度
print("\n🎉 步骤5: 查看升级后的额度")
quota = gateway.get_user_quota(new_user)
print(f"  订阅等级: {quota.tier.value}")
print(f"  消息额度: {quota.messages_limit}")
print(f"  图片额度: {quota.images_limit}")
print(f"  订阅有效期: {quota.subscription_end}")

print("\n" + "=" * 50)
print("演示完成！")